<a href="https://colab.research.google.com/github/haru1489248/nlp-100-nock/blob/main/ch09/section_89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 89. アーキテクチャの変更
88のファインチューニングのアーキテクチャを変更する。今回は最大値プーリングを用いる
### 最大値プーリングとは
文中の各トークンが持つ隠れ状態ベクトルのうち、各次元（要素）ごとに最大の値だけを集めて1つの文ベクトルにする方法。



In [ ]:
!pip install -U transformers evaluate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModel,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
import evaluate # hugging face公式のライブラリ

In [ ]:
class SST2Dataset(Dataset):
  def __init__(self, sentences, labels, tokenizer):
    super().__init__()
    self.encodings = tokenizer(sentences, truncation=True) # paddingはcollator側でやる
    self.labels = labels

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    # items()はPythonのdictのメソッド。
    # encodingsはdict互換のBatchEncodingオブジェクトなのでitems()が使える
    item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
    item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
    return item

- dropout: モデルの過学習を抑えるために学習中だけランダム（今回は10%の確率）にベクトルの要素を0にする
- nn.CrossEntropyLoss(): 内部でsoftmaxまでやってくれる損失関数（クロスエントロピー損失）
- nn.CrossEntropyLoss()が期待する入力の形
```
入力(logits) : (N, C)
正解ラベル : (N)
N = サンプル数
C = クラス数
```
- view: PyTorchでテンソル（shape）の形を変えるためのメソッド   
コードの例:
```
x = torch.tensor([[1,2,3],[4,5,6]])
x.shape  # [2,3]

x2 = x.view(3,2)
x2
# [[1,2],
#  [3,4],
#  [5,6]]

要素数N個で、
x.view(-1, 2)
と書いたら
-1の次元は
N / 2となる。
```
- masked_fill(condition, value)
  - 意味はconditionがTrueの位置をvalueで埋める
  - 今回は~maskがTrueの位置を-∞で埋める
  - ~でTrueをFalseにFalseをTrueに変換する
- なぜ-inf?
  - max poolingを考えると-infは絶対に選ばれなくなるため

In [ ]:
class MaxPoolingClassifier(nn.Module):
  def __init__(self, model_name, num_labels=2):
    super().__init__()
    self.bert = AutoModel.from_pretrained(model_name)
    self.dropout = nn.Dropout(0.1)
    self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

  def forward(self, input_ids, attention_mask, labels=None):
    # attention_maskは内部で使われるが、出力にPADも残るので
    # pooling前に自分で除外する必要がある
    outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)

    # PADトークンも返ってくるのでmaskする
    mask = attention_mask.unsqueeze(-1).bool()
    last_hidden = outputs.last_hidden_state.masked_fill(~mask, float("-inf"))

    # max(dim=1)の戻り値 = values, indicies
    pooled_output = last_hidden.max(dim=1).values

    pooled_output = self.dropout(pooled_output)
    logits = self.classifier(pooled_output)

    loss = None
    if labels is not None:
      loss_fct = nn.CrossEntropyLoss()

      # 今回はたまたまCrossEntropyLoss()の求めているshapeにあっていたため
      # そのまま入力している
      loss = loss_fct(logits, labels)
      # loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))

    return {"loss": loss, "logits": logits}

In [ ]:
def load_SST2(path):
  sentences, labels = [], []
  with open(path, 'r') as f:
    reader = csv.DictReader(f, delimiter='\t')
    for row in reader:
      sentences.append(row['sentence'])
      labels.append(int(row['label']))
  return sentences, labels

### compute_matricsとは？
評価時に使う指標（ここでは正解率）を計算する関数
- eval_predとは?
  - tupleで中身は(logits: モデルの出力, labels: 正解ラベル)となっている
  - logits.shape: (N, num_labels)
    - 各サンプルに対するクラスごとのスコア
  - labels.shape: (N)
    - 正解ラベル(0 or 1)
- logitsとは？
  - BERTの分類モデルは`outputs = model(...)`; `outputs.logits`を返す
  - 例（2クラス）:

```
logits = [
  [2.3, -0.8], # sample 1
  [-1.1, 0.4], # sample 2
]
```
  - softmax前のスコア
  - 大きい方がモデルの予測クラス
- `np.argmax(logits, axis=-1)`とは？
  - 各サンプルについて一番スコアが高いクラスのインデックスを取る
  ```
  [2.3, -0.8] -> 0
  [-1.1, 0.4] -> 1
  ```



In [ ]:
accuracy = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  preds = np.argmax(logits, axis=-1)
  return accuracy.compute(predictions=preds, references=labels)

### TrainingArgumentsとは？
trainerに渡す学習の設定
- output_dir: 学習結果の保存先ディレクトリ
- eval_strategy: いつ評価するか
  - `"epoch"`: 1エポック終わるごとに評価
  - `"steps"`: 一定ステップごと
  - `"no"`: 評価しない
- save_strategy: いつモデルを保存するか
- learning_rate: 学習率（2e-5=2*10^{-5}）
- per_device_train_batch_size: 1GPU（or CPU）あたりの学習サイズ
- per_device_eval_batch_size: 評価時のバッチサイズ（勾配計算しないので、学習時より大きくてもいい）
- num_train_epochs: データを何周するか
- weight_decay: L2正則化。重みが大きくなすぎるのを防ぐ（正則化項の係数）
- loging_steps: 何ステップごとにログを出すか（lossやlearning_rateを表示）
1ステップ=一回のoptimizer更新（だいたい1バッチ処理）

In [ ]:
def main():
  model_id = "bert-base-uncased"
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  model = MaxPoolingClassifier(model_id)

  data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # batchごとに最大長に合わせてpaddingしてくれるcollator

  train_src = '/content/drive/MyDrive/SST-2/train.tsv'
  dev_src = '/content/drive/MyDrive/SST-2/dev.tsv'
  train_sentences, train_labels = load_SST2(train_src)
  dev_sentences, dev_labels = load_SST2(dev_src)

  train_dataset = SST2Dataset(train_sentences, train_labels, tokenizer)
  dev_dataset = SST2Dataset(dev_sentences, dev_labels, tokenizer)

  training_args = TrainingArguments(
    output_dir="sst2-bert",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=50,
  )

  trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
  )

  trainer.train()

  eval_results = trainer.evaluate() # compute_metricsを呼ぶ
  print(f"最終的な評価結果: {eval_results}")

In [ ]:
if __name__ == '__main__':
  main()